##### 파일 명세
- udrt_kakao_cdh_old_new_place_category 컬렉션의 uuid 기준으로 mapped_category count

In [1]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from collections import defaultdict

In [2]:
# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))
db = client.get_database(os.getenv('DB_Collection'))

#컬렉션 불러오기
cl_udrt_kakao_cdh_old_new_place_category=db.get_collection("udrt_kakao_cdh_old_new_place_category")

In [4]:
# uuid를 기준으로 mapped_categroy의 값이 몇번 등장하는지 빈도수 조사

count_dict=defaultdict(lambda: defaultdict(int))

for doc in cl_udrt_kakao_cdh_old_new_place_category.find({},{"uuid":1, "mapped_category":1, "_id":0}):
    count_dict[doc["uuid"]][doc["mapped_category"]]+=1

new_documents=[]
for uuid, dict_mapped_category_count in count_dict.items():
    for mapped_category, count in dict_mapped_category_count.items():
        new_documents.append({
            "uuid":uuid,
            "mapped_category": mapped_category,
            "frequency": count
        })

# 최종 데이터를 new_collection에 삽입
new_collection=db["mapped_category_frequency"]
if new_documents:
    new_collection.insert_many(new_documents)

print(f"{len(new_documents)}개의 문서를 mapped_category_frequency 삽입 완료.")

82344개의 문서를 mapped_category_frequency 삽입 완료.
